In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from scipy import stats
 
from data_bpm.ml_logic.preprocessor import preprocess_features
from data_bpm.ml_logic.registry import load_preproc_pipeline

from sklearn.svm import SVC 
from sklearn.model_selection import RandomizedSearchCV, cross_validate

In [3]:
data = pd.read_csv('../raw_data/data_for_ml.csv',index_col=0)

In [4]:
data.head()

,description,headline,location,company,jobTitle,jobDescription,jobLocation,jobDateRange,jobDuration,company2,...,endorsement2,skill3,endorsement3,skill4,endorsement4,skill5,endorsement5,skill6,endorsement6,Attendance
UserID,,,,,,,,,,,,,,,,,,,,,
512.0,Experienced professional with the ability to c...,"General Manager - US Finance, Legal, HR, Accou...","Carrollton, Texas, United States",orbis systems,"General Manager, Orbis International Technolog...",A highly driven professional leading the US op...,"Carrollton, Texas, United States",May 2022 - Present,1 yr 10 mos,Texas Instruments,...,21.0,Strategic Planning,21.0,Forecasting,21.0,Organizational Development,21.0,Business Strategy,21.0,0
652.0,"I am a Product Owner at World Simplified UG, a...",Tech product support intern at Holidu,"Berlin, Berlin, Germany",holidu,Tech product support intern,NaN,"Munich, Bavaria, Germany · Hybrid",Oct 2023 - Present,5 mos,World Simplified UG,...,NaN,Team Leadership,NaN,Cross-functional Team Leadership,1.0,Project Planning,21.0,Business Development,NaN,0
618.0,Designer & Problem Solver | CS Student | Aimin...,Making Interfaces Sexier - UI/UX/Dev,"Berlin, Berlin, Germany",a.t.e.m. management gmbh,Web Designer,Skills: Web Design · Cascading Style Sheets (C...,"Berlin, Germany",Aug 2022 - Present,1 yr 7 mos,Freelance,...,NaN,JavaScript,2.0,Cascading Style Sheets (CSS),2.0,Front-End Development,1.0,Wireframing,1.0,0
96.0,NaN,Product & Growth,"Berlin, Berlin, Germany",clink,Chief Experience Officer,NaN,"Berlin, Germany",Sep 2023 - Present,6 mos,OptioPay Group (now Clink),...,40.0,Cross-functional Team Leadership,3.0,Business Development,33.0,Mobile Applications,10.0,Mobile Technology,35.0,0
194.0,As a product marketing manager at the JetBrain...,Product marketing manager at JetBrains | Mentor,"Berlin, Berlin, Germany",jetbrains,Product Marketing Manager,NaN,"Berlin, Germany",Oct 2021 - Present,2 yrs 5 mos,Product Marketing Alliance,...,8.0,Marketing Communications,8.0,Customer Insight,13.0,Market Analysis,12.0,Marketing Research,11.0,0


In [5]:
data['Attendance'].value_counts()


Attendance
0    265
1    204
Name: count, dtype: int64

In [16]:
y_train = data['Attendance']
X_train = data.drop(columns = ['Attendance'])

X_processed = preprocess_features(X_train)
X_processed

✅ X_processed, with shape (469, 19)

Saving preprocessing pipeline from local disk...
✅ Preprocessing pipeline saved locally


array([[1.        , 0.07942238, 0.48854962, ..., 0.28560673, 0.5012473 ,
        0.37360172],
       [1.        , 0.01805054, 0.01908397, ..., 0.55462501, 0.48195116,
        0.48788221],
       [1.        , 0.06859206, 0.12977099, ..., 0.53936   , 0.3589593 ,
        0.48345559],
       ...,
       [1.        , 0.02527076, 0.03053435, ..., 0.35047975, 0.46681548,
        0.36767028],
       [1.        , 0.09747292, 0.1870229 , ..., 0.3964628 , 0.43520686,
        0.41195238],
       [1.        , 0.07581227, 0.09923664, ..., 0.56068655, 0.51723509,
        0.41072087]])

In [7]:
X_processed.shape

(469, 19)

In [8]:
y_train.shape

(469,)

In [9]:
model = SVC()

grid = { 'kernel' : ['linear', 'rbf', 'sigmoid'],
        'C' : [0.01, 0.1, 1, 10, 100],
        'gamma' : stats.uniform()    
}

randsearch = RandomizedSearchCV(estimator=model, param_distributions=grid, n_iter=500, scoring='accuracy', n_jobs=-1)

randsearch.fit(X_processed, y_train)

randsearch.best_score_

0.5757263784031115

In [10]:
randsearch.best_params_


{'C': 100, 'gamma': 0.21658780941701816, 'kernel': 'rbf'}

In [11]:
final_model = randsearch.best_estimator_

In [12]:
X_pred = pd.read_csv('../raw_data/predict.csv')
X_pred

,headline,description,jobTitle,jobDescription,jobDuration,jobDateRange,jobTitle2,jobDuration2,schoolDateRange,skill1,skill2,skill3
0,"Product Leader, Sustainability enthusiast",Bridging the world of technology and business ...,Head of Product,Spearhead product development and strategy for...,1 yr,Aug 2022 - Jul 2023,Product Lead,8 mos,Sep 2023 - Nov 2023,Management Consulting,Competitive Landscape,Stakeholder Management


In [18]:
preproc_pipeline = load_preproc_pipeline()
X_pred_processed = preproc_pipeline.transform(X_pred)
X_pred_processed


Load preprocessing pipeline from local disk...
✅ Preprocessing pipeline loaded from local disk


/home/shubhi/.pyenv/versions/3.10.6/envs/data-bpm/lib/python3.10/site-packages/sklearn/decomposition/_truncated_svd.py:273: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = exp_var / full_var


ValueError: X has 5 features, but MinMaxScaler is expecting 19 features as input.

In [ ]:
final_model.predict_proba(X_pred_processed)